In [1]:
import glob
import time
import numpy as np
np.set_printoptions(suppress=True)

from SmoothSparseWeights_CVXOPT import *

In [2]:
# exampleName="cheburashka"
# poses_folder="poses-2"
# poses_ind=7
# clip_k=4
# handle_num=11


exampleName="cow"
poses_folder="poses-2"
poses_ind=5
clip_k=4
handle_num=15


# exampleName="wolf"
# poses_folder="poses-1"
# poses_ind=2
# clip_k=4
# handle_num=21



weights = {'W_spatial': 0.0}

In [3]:
#### read obj file

### rest_poses:
vertices_obj_path="./models/"+exampleName+"-all/"+exampleName+".obj"
mesh0=TriMesh.FromOBJ_FileName(vertices_obj_path)

### other poses
vertices_obj_dir="./models/"+exampleName+"-all/"+poses_folder+"/"
filenames=glob.glob(vertices_obj_dir+"*.obj")
print(filenames)

mesh1_list=[]
for i in range(len(filenames)):
    mesh1_list.append(TriMesh.FromOBJ_FileName(filenames[i]))


vertices1=vertices1_temp=np.hstack((np.asarray(mesh0.vs),np.ones((len(mesh0.vs),1))))
vertices2=vertices2_temp=np.asarray(mesh1_list[0].vs)
for i in range(1,len(mesh1_list)):
    vertices1=np.hstack((vertices1, vertices1_temp))
    vertices2=np.hstack((vertices2, np.asarray(mesh1_list[i].vs)))
print(vertices1.shape) 
print(vertices2.shape) 


['./models/cow-all/poses-2/cow-1.obj', './models/cow-all/poses-2/cow-2.obj', './models/cow-all/poses-2/cow-3.obj', './models/cow-all/poses-2/cow-4.obj', './models/cow-all/poses-2/cow-5.obj', './models/cow-all/poses-2/cow-6.obj', './models/cow-all/poses-2/cow-7.obj']
(11666, 28)
(11666, 21)


In [4]:
#### read our output of simplex_hull.py 
results_path="./results_pipeline/"+exampleName+"-"+str(handle_num)+"/result.txt"
endmembers, mixing_weights=load_result(results_path)
### endmembers shape is handle * poses * 12??

# ### this is to make sure the 3*4 or 4*3 error does not happen.
# endmembers=endmembers.reshape((-1, 4, 3))
# endmembers=np.transpose(endmembers, (0,2,1))
# endmembers=endmembers.reshape((len(mixing_weights), -1, 12))


endmembers=endmembers.reshape((len(mixing_weights),-1))
mixing_weights=mixing_weights.T


print (endmembers.shape)
print (mixing_weights.shape)
print (mixing_weights[:3])
mixing_weights_clip, grad_zero_indices=clip_first_k_values(mixing_weights, clip_k)
print (mixing_weights_clip[:3])
print ((col_major(mixing_weights_clip)[grad_zero_indices]).sum()) ### should be zero

(15, 84)
(11666, 15)
[[0.20570602 0.00042219 0.02655054 0.15167153 0.00018981 0.00004767
  0.00030539 0.00002423 0.02001831 0.00000105 0.00102998 0.14445015
  0.00045419 0.44910447 0.00002447]
 [0.21555833 0.0004373  0.02679411 0.16466939 0.00016431 0.00004939
  0.00029969 0.00002367 0.01424946 0.00000119 0.00101018 0.15187666
  0.00048589 0.42435294 0.00002749]
 [0.24024097 0.00040106 0.02997077 0.15734726 0.00013956 0.00004676
  0.00030475 0.00002278 0.01438586 0.00000122 0.00103343 0.11445163
  0.00051674 0.44110425 0.00003297]]
[[0.20570602 0.         0.         0.15167153 0.         0.
  0.         0.         0.         0.         0.         0.14445015
  0.         0.44910447 0.        ]
 [0.21555833 0.         0.         0.16466939 0.         0.
  0.         0.         0.         0.         0.         0.15187666
  0.         0.42435294 0.        ]
 [0.24024097 0.         0.         0.15734726 0.         0.
  0.         0.         0.         0.         0.         0.11445163
  0.  

In [5]:
scale=find_scale(vertices1)
print (scale)

708.6870049193498


In [6]:
recovered_vertices2=recover_vertices(mixing_weights, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

0.07533573276941447


In [7]:
recovered_vertices2=recover_vertices(mixing_weights_clip, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

initials={'x': col_major(mixing_weights_clip.reshape((-1,len(endmembers)))).reshape((-1,1))}
print (initials['x'].shape)

start=time.time()

res=run(mesh0, mesh1_list, weights, endmembers, mixing_weights.ravel(), grad_zero_indices, initials=initials, choice="transformation_error")

res=res.reshape(mixing_weights.shape)
end=time.time()
print("using time: ", end-start)

recovered_vertices2=recover_vertices(res, vertices1, endmembers)
print (E_RMS_kavan2010(vertices2, recovered_vertices2, scale))

37144.77739542564
(174990, 1)
(11666, 28)
(11666, 21)
11666 7 15
W_spatial 0.0
     pcost       dcost       gap    pres   dres
 0: -1.0742e+08 -1.0873e+08  4e+07  2e+01  2e-13
 1: -1.0708e+08 -1.0846e+08  2e+07  6e+00  2e-13
 2: -1.0642e+08 -1.0780e+08  1e+07  3e+00  1e-13
 3: -1.0600e+08 -1.0758e+08  8e+06  1e+00  1e-13
 4: -1.0572e+08 -1.0724e+08  6e+06  8e-01  1e-13
 5: -1.0571e+08 -1.0724e+08  6e+06  8e-01  1e-13
 6: -1.0550e+08 -1.0673e+08  4e+06  4e-01  1e-13
 7: -1.0542e+08 -1.0646e+08  3e+06  3e-01  1e-13
 8: -1.0530e+08 -1.0614e+08  2e+06  2e-01  1e-13
 9: -1.0527e+08 -1.0595e+08  2e+06  2e-01  1e-13
10: -1.0525e+08 -1.0579e+08  2e+06  1e-01  1e-13
11: -1.0519e+08 -1.0555e+08  2e+06  1e-01  1e-13
12: -1.0511e+08 -1.0524e+08  2e+06  1e-01  1e-13
13: -1.0503e+08 -1.0497e+08  2e+06  1e-01  1e-13
14: -1.0485e+08 -1.0459e+08  2e+06  9e-02  1e-13
15: -1.0457e+08 -1.0418e+08  2e+06  7e-02  1e-13
16: -1.0421e+08 -1.0388e+08  2e+06  5e-02  1e-13
17: -1.0384e+08 -1.0366e+08  2e+06  4e-0

In [8]:
print (res.reshape((-1, len(endmembers)))[:3]) ### should have same zeros as initial input.
print ((col_major(res)[grad_zero_indices]).sum()) ### should be zero
print ((col_major(res)[grad_zero_indices]).max()) ### should be zero

[[ 0.0488357  -0.00025218 -0.00049711  0.85894611 -0.00042451 -0.00019948
   0.00003066  0.00020723 -0.00053548  0.00069607  0.00026473  0.04839043
  -0.00035479  0.04373759  0.00115504]
 [ 0.04865433 -0.00023534 -0.00048779  0.85893776 -0.00040002 -0.00022618
  -0.00009843  0.00020417 -0.00054918  0.00069575  0.00035076  0.0485271
  -0.00032414  0.04379297  0.00115825]
 [ 0.04856603 -0.00030226 -0.00046907  0.858842   -0.00040621 -0.00025675
  -0.00015609  0.0002012  -0.00055037  0.00069594  0.00048723  0.04853769
  -0.00031577  0.04394801  0.00117843]]
1.5993784074908035e-10
0.0043737005785695355


In [9]:
print (((1-res.sum(axis=-1))**2).sum()/len(res))
print (res.min(), res.max())

7.783510782681785e-27
-0.0006422016771267755 0.9115699649712768


In [10]:
recover_diff= (res.dot(endmembers)-mixing_weights.dot(endmembers))
recover_diff= (recover_diff**2).sum()
print (recover_diff)

15196379513975.172


In [11]:
# #### read gt per vertex transformation 
# groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+"-*.DMAT")
# print(groundtruth_names)

# gt_all=None
# for i in range(len(groundtruth_names)):
#     gt=load_DMAT(groundtruth_names[i])
#     gt=gt.T
#     gt=gt.reshape((-1,4,3))
#     gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
#     if gt_all is None:
#         gt_all=gt
#     else:
#         gt_all=np.hstack((gt_all, gt))
    
# print(gt_all.shape)


In [12]:
#### read gt per vertex mixing weights 
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+exampleName+".DMAT")
print(groundtruth_names)
gt=load_DMAT(groundtruth_names[0])
gt_mixing_weights=gt.T.copy()
print (gt_mixing_weights.shape)

print (((gt_mixing_weights.sum(axis=-1)-1)**2).sum())

['./models/cow-all/cow.DMAT']
(11666, 15)
1.5653958587979453e-28


In [13]:
### load groundtruth endmember
groundtruth_names=glob.glob("./models/"+exampleName+"-all/"+poses_folder+"/"+exampleName+"-*.Tmat")
print(groundtruth_names)

gt_handles=None
for i in range(len(groundtruth_names)):
    gt=load_Tmat(groundtruth_names[i])
    gt=gt.reshape((-1,4,3))
    gt=np.transpose(gt, (0,2,1)).reshape((-1,12))
    if gt_handles is None:
        gt_handles=gt
    else:
        gt_handles=np.hstack((gt_handles, gt))
    
print(gt_handles.shape)
gt_handles=gt_handles.reshape(len(groundtruth_names), -1, 12)

gt_handles=np.transpose(gt_handles.reshape(-1, len(groundtruth_names), 12), (1,0,2)) ### is this true?

print(gt_handles.shape)

for i in range(len(gt_handles)):
    print (gt_handles[i][2].reshape((3,4)))

test=gt_handles[1][2].reshape((3,4))[:,:3]
print (test.T.dot(test))

['./models/cow-all/poses-2/cow-1.Tmat', './models/cow-all/poses-2/cow-2.Tmat', './models/cow-all/poses-2/cow-3.Tmat', './models/cow-all/poses-2/cow-4.Tmat', './models/cow-all/poses-2/cow-5.Tmat', './models/cow-all/poses-2/cow-6.Tmat', './models/cow-all/poses-2/cow-7.Tmat']
(15, 84)
(7, 15, 12)
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.998956    0.0322376   0.0323555  -2.24837   ]
 [-0.0456047   0.664913    0.745527   27.1422    ]
 [ 0.00252045 -0.746225    0.665689   53.4901    ]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]]
[[ 0.99999923 -0.00000004  0.00000002]
 [-0.00000004  1.00000031 -0.00000012]
 [ 0.00000002 -0.00000012  0.99999923]]


In [31]:
Vertices=vertices2[:,poses_ind*3:poses_ind*3+3]

## use recovered vertices instead.
recovered_vertices2=recover_vertices(res2, vertices1, endmembers).reshape((len(vertices1),-1))
Vertices=recovered_vertices2[:,poses_ind*3:poses_ind*3+3]


In [32]:
print (res.sum(axis=0))
print (res.T.dot(Vertices))

res2=res
res2=res.clip(0,1) ### this is important, when estimate handle position later.
print (((1-res2.sum(axis=-1))**2).sum()/len(res2))

print (res2.sum(axis=0))
print (res2.T.dot(Vertices))

[ 670.2643694     0.63173762  739.75593732 9028.50360065   -3.90749162
   -1.82023661    0.02634296    2.62539863   47.93159462    7.76160885
   -2.70231202  589.26563627   -1.24419999  576.75633724   12.15167668]
[[   676479.34981704   -431417.97732414     21503.8137028 ]
 [     -687.87545346      1120.00041222      -192.61510175]
 [ -1227436.62574041   1835023.70558569   5240164.03736247]
 [-11096625.44884238  23148140.25531221  24088450.23492969]
 [     4115.71521381     -8925.12642405    -10036.42163306]
 [     1346.33686403     -3529.12752935     -4107.65641221]
 [    -1013.8552239       1623.70769166     -2784.51020272]
 [    -2269.02912945      4912.10380954      7206.19482757]
 [   -81623.1529577     122312.20207824    317699.31117342]
 [   -11003.95074331     21619.91848198     24688.58872559]
 [     3630.43327626     -6828.85860072     -3896.34650231]
 [   401097.22589322    -85139.9718273     134955.06533218]
 [    14743.74640716    -21644.27465504    -20541.73320832]
 [   2

In [34]:
### plot handles corresponding to gt mixing weights, recovered mixing weights, and post-processed mixing weights.
###

eps=1e-30

gt_handle_position=gt_mixing_weights.T.dot(Vertices)/(gt_mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
recovered_handle_position=mixing_weights.T.dot(Vertices)/(mixing_weights.T.sum(axis=-1)[...,np.newaxis]+eps)
cliped_handle_position=mixing_weights_clip.T.dot(Vertices)/(mixing_weights_clip.T.sum(axis=-1)[...,np.newaxis]+eps)
postprocessed_handle_position=res2.T.dot(Vertices)/(res2.T.sum(axis=-1)[...,np.newaxis]+eps)


%matplotlib 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig= plt.figure()
ax=fig.add_subplot(111, projection='3d')

ax.scatter(gt_handle_position[:,0],gt_handle_position[:,1], gt_handle_position[:,2], c='k', s=50)
ax.scatter(recovered_handle_position[:,0],recovered_handle_position[:,1], recovered_handle_position[:,2], c='r', s=50)
# ax.scatter(cliped_handle_position[:,0],cliped_handle_position[:,1], cliped_handle_position[:,2], c='g', s=50)
ax.scatter(postprocessed_handle_position[:,0],postprocessed_handle_position[:,1], postprocessed_handle_position[:,2], c='b', s=50)
ax.scatter(Vertices[:,0],Vertices[:,1], Vertices[:,2], color='y', s=1, edgecolors='none')

plt.show()

Using matplotlib backend: MacOSX
